In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from sklearn.metrics import matthews_corrcoef
import csv

In [3]:
from fastai import *
from fastai.vision import *

In [4]:
from data_loader import ImageItemListCell

In [5]:
path = Path('/data/Datasets/WhiteBloodCancer/train/')
path_submission = Path('/data/Datasets/WhiteBloodCancer/train/submission/')
path_models = Path('/data/Datasets/WhiteBloodCancer/train/rn-18-RND/')
test_path = Path('/data/Datasets/WhiteBloodCancer/test/')

test_files = get_files(test_path)
test = ImageItemList(test_files)

test_files[:10]

[PosixPath('/data/Datasets/WhiteBloodCancer/test/1066.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/710.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1384.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/996.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1604.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/232.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1302.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/628.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1467.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/905.bmp')]

In [6]:
for model in path_models.glob('*.pkl'):

    learn = load_learner(path, model, test)
    learn.model.float()
    
    # Use TTA
    #tta_result, y_test_tta = learn.TTA(ds_type=DatasetType.Test)
    #y_pred = np.argmax(tta_result, axis=1)
    
    # use normal prediction
    preds_test,y_test=learn.get_preds(ds_type=DatasetType.Test)
    y_pred = torch.sigmoid(preds_test)
    
    with open(path_submission.joinpath('{}.csv'.format(model.stem)), mode='w') as submission:
        submission_writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for fn, y in zip(learn.data.test_dl.items, to_np(y_pred)):
            submission_writer.writerow([fn.name, y[0], y[1]])

IndexError: invalid index to scalar variable.

In [ ]:
ensemble_ALL = np.zeros(shape=(len(learn.data.test_dl.items)))
ensemble_Hem = np.zeros(shape=(len(learn.data.test_dl.items)))
model_count = len(list(path_submission.glob('*.csv')))
for submission in path_submission.glob('*.csv'):

    with open(submission, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for i, row in enumerate(reader):
            ensemble_ALL[i] += float(row[1])
            ensemble_Hem[i] += float(row[2])


ensemble_ALL = ensemble_ALL / model_count
ensemble_Hem = ensemble_Hem / model_count


submission = [0 for i in range(ensemble_ALL.shape[0])]

threshold = 0.5 #0.65
for fn, y in zip(learn.data.test_dl.items, ensemble_ALL):
    index = int(fn.name.replace(".bmp",'')) - 1
    y = 1 if y > threshold else 0
    submission[index] = y

with open(path_submission.joinpath('isbi_valid.predict'), 'w') as f:
    for item in submission:
        f.write("{}\n".format(item))